In [9]:
sample_list = [1, 2, 3, 4, 5]

for idx, element in enumerate(sample_list):
    element +=1
    sample_list[idx] = element

print(sample_list)

[2, 3, 4, 5, 6]


In [1]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import plot_tree

from sklearn.model_selection import train_test_split
# Regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
# Clasification
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, precision_score, accuracy_score
# Best_searches
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
# Standarization
from sklearn.preprocessing import StandardScaler

from keras import models
from keras.layers import Dense

In [2]:
df_train = pd.read_csv('https://raw.githubusercontent.com/Mateusz-Matejko/kaggle/main/projects/competition-clasification-binary-of-machine-failures/data/train.csv', encoding='utf8')
df_test = pd.read_csv('https://raw.githubusercontent.com/Mateusz-Matejko/kaggle/main/projects/competition-clasification-binary-of-machine-failures/data/test.csv', encoding='utf8')
df_final = pd.read_csv('https://raw.githubusercontent.com/Mateusz-Matejko/kaggle/main/projects/competition-clasification-binary-of-machine-failures/data/sample_submission.csv', encoding='utf8')

In [3]:
df_train.describe()

,id,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
count,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000
mean,68214.000000,299.862776,309.941070,1520.331110,40.348643,104.408901,0.015744,0.001554,0.005160,0.002397,0.003958,0.002258
std,39383.804275,1.862247,1.385173,138.736632,8.502229,63.965040,0.124486,0.039389,0.071649,0.048899,0.062789,0.047461
min,0.000000,295.300000,305.800000,1181.000000,3.800000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,34107.000000,298.300000,308.700000,1432.000000,34.600000,48.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,68214.000000,300.000000,310.000000,1493.000000,40.400000,106.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,102321.000000,301.200000,310.900000,1580.000000,46.100000,159.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,136428.000000,304.400000,313.800000,2886.000000,76.600000,253.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [4]:
df_test[df_test['PWF'] != 0]

,id,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],TWF,HDF,PWF,OSF,RNF
405,136834,L47380,L,298.4,308.6,2636,12.8,103,0,0,1,0,0
576,137005,L56355,L,297.9,309.0,1403,64.2,0,0,0,1,0,0
950,137379,L49935,L,299.7,309.3,1478,38.8,8,0,0,1,0,0
1570,137999,L55609,L,299.0,310.4,2861,4.6,92,0,0,1,0,0
2653,139082,L53074,L,301.2,311.0,1267,69.3,164,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
86883,223312,M20301,M,303.0,312.6,1363,62.9,119,0,0,1,0,0
87045,223474,L49512,L,299.2,308.5,2232,15.9,220,0,0,1,0,0
89247,225676,M18633,M,302.4,311.1,2659,10.8,139,0,0,1,0,0
90239,226668,L51961,L,303.4,311.7,1374,63.7,121,0,0,1,0,0


In [5]:
df_train.shape

(136429, 14)

In [6]:
df_train[df_train['Machine failure'] != 0].shape

(2148, 14)

## Opis danych: 

id - 
    Wiemy o co chodzi
    
Podcut ID - 
    Pomysły: Zmapować konkretny produkt id, jako taki z dużą lub małą szansą na uszkodzenie. 
    
### Zrobione 
Type - ( H, M, L )(High, Medium, Low) - 
    Im wyższy tym mniejsza szansa na uszkodzenie i większa tolerancja na długotrwałą pracę


Air temparture - temperatura powietrza
    zakres: 295 - 304 stopnie, median 300
    Różnica między tą temperaturą a temperaturą procesu może wskazywać na przegrzewanie.
    
### Zrobione
<i>Pomysły: Zakodować różnicę między temperaturami.</i>
    
Process temperature - temperatura pracy
    zakres 305 - 314, median 310.
    Pomysły: Sprawdzić zależność między awariami a tą temperaturą, DLA WSZYSTKICH TYPU AWARII

### Zrobione 
<i> Note: ZAPYTAĆ KACPRA CZY WARTO SPRAWDZIĆ TO NA WARUNKU + 0.5C </i>
<i>Jeżeli Air Temprature będzie równe Process temp lub większe, użądzenie nie będzie się w stanie schłodzić i się nagrzeje.</i> 

### In progress
Rotational speed rpm - prędkość rotacji/obrotu
    zakres 1181 - 2886, 996 unikalnych
    Pomysł: Porównać z Tool Wear 
    
Torque Nm - Moment obrotowy 
    Pomysł: użyć wzór, spróbować oszacować błąd. Power (kW) = Torque (N.m) x Speed (RPM) z TWF
    Możliwe: powiązanie OSF
    
Tool wear - Czas sesji użycia narzędzia
    Zakres 0 - 253
    Pomysł: Im większy tool wear, może powodować awarie
    
Typy awarii:
    TWF - Zużycie
    HDF - Przegrzanie
    PWF - Za mała moc
    OSF - Przeciążenie mechaniczne
    RNF - Losowe
    Pomysły: Zobaczyć zależności pomiędzy tymi typami awarii i dokładnie je zbadać.
    
Zbadać: outliersy dlaczego min czas sesji = 0

In [7]:
df_train.describe(include='all')

,id,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
count,136429.000000,136429,136429,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000
unique,NaN,9976,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,L53257,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,139,95354,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,68214.000000,NaN,NaN,299.862776,309.941070,1520.331110,40.348643,104.408901,0.015744,0.001554,0.005160,0.002397,0.003958,0.002258
std,39383.804275,NaN,NaN,1.862247,1.385173,138.736632,8.502229,63.965040,0.124486,0.039389,0.071649,0.048899,0.062789,0.047461
min,0.000000,NaN,NaN,295.300000,305.800000,1181.000000,3.800000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,34107.000000,NaN,NaN,298.300000,308.700000,1432.000000,34.600000,48.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,68214.000000,NaN,NaN,300.000000,310.000000,1493.000000,40.400000,106.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,102321.000000,NaN,NaN,301.200000,310.900000,1580.000000,46.100000,159.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [16]:
df_train.isnull().sum().sum()

0

### Power (kW) = Torque (N.m) x Speed (RPM)

## Get to know your data

In [9]:
df_train['Product ID'].unique().shape #(9976,)
df_train['Type'].unique() #array(['L', 'M', 'H'], dtype=object)
df_train['Air temperature [K]'] #(95,)
df_train['Process temperature [K]'].unique().shape #(81,)
df_train['Tool wear [min]'].unique().shape #(246,)
df_train['Rotational speed [rpm]'].unique().shape #(952,)
df_train['Torque [Nm]'].unique().shape #(611,)
df_train['Tool wear [min]'].unique().shape #(246,)

(246,)

In [10]:
# Dataset shape
print(df_train.shape, "\n") # (74051, 10)

# Find out any Null values
df_train.isnull().sum() # 0 null values

(136429, 14) 



id                         0
Product ID                 0
Type                       0
Air temperature [K]        0
Process temperature [K]    0
Rotational speed [rpm]     0
Torque [Nm]                0
Tool wear [min]            0
Machine failure            0
TWF                        0
HDF                        0
PWF                        0
OSF                        0
RNF                        0
dtype: int64

In [11]:
df_train.head()

,id,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,0,L50096,L,300.6,309.6,1596,36.1,140,0,0,0,0,0,0
1,1,M20343,M,302.6,312.1,1759,29.1,200,0,0,0,0,0,0
2,2,L49454,L,299.3,308.5,1805,26.5,25,0,0,0,0,0,0
3,3,L53355,L,301.0,310.9,1524,44.3,197,0,0,0,0,0,0
4,4,M24050,M,298.0,309.0,1641,35.4,34,0,0,0,0,0,0


In [12]:
df_train.groupby('Type')['Machine failure'].mean()

Type
H    0.013000
L    0.016727
M    0.013592
Name: Machine failure, dtype: float64

In [13]:
type_dict = {
    'H': 0.013000,
    'M': 0.013592,
    'L': 0.016727
}

<i>Jeżeli Air Temprature będzie równe Process temp lub większe, użądzenie nie będzie się w stanie schłodzić i się nagrzeje.</i> 

1. Potrzebne dane:
    - Air temprature = 6
    - Process temperature = 4
2. Porównanie:
    - 

In [81]:
df_train = pd.read_csv('https://raw.githubusercontent.com/Mateusz-Matejko/kaggle/main/projects/competition-clasification-binary-of-machine-failures/data/train.csv', encoding='utf8')

def process_data(df:pd.DataFrame):
    df['Power [KW]'] = df['Torque [Nm]'] * df['Rotational speed [rpm]']
    df['Type'] = df['Type'].map(type_dict)
    df['Air temperature [C]'] = df['Air temperature [K]'] - 272.15
    df['Process temperature [C]'] = df['Process temperature [K]'] - 272.15
    df = df.drop(labels=['Air temperature [K]', 'Process temperature [K]'], axis=1)
    print(df.columns)
    df['Possbile_overheating'] = (df['Air temperature [C]'] >= df['Process temperature [C]']) #possibly add sth to C value
    df['Temperature difference'] = df['Process temperature [C]'] - df['Air temperature [C]']

    return df

In [82]:
df_train=process_data(df_train)

Index(['id', 'Product ID', 'Type', 'Rotational speed [rpm]', 'Torque [Nm]',
       'Tool wear [min]', 'Machine failure', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF',
       'Power [KW]', 'Air temperature [C]', 'Process temperature [C]'],
      dtype='object')


In [83]:
df_train.head()

,id,Product ID,Type,Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF,Power [KW],Air temperature [C],Process temperature [C],Possbile_overheating,Temperature difference
0,0,L50096,0.016727,1596,36.1,140,0,0,0,0,0,0,57615.6,28.45,37.45,False,9.0
1,1,M20343,0.013592,1759,29.1,200,0,0,0,0,0,0,51186.9,30.45,39.95,False,9.5
2,2,L49454,0.016727,1805,26.5,25,0,0,0,0,0,0,47832.5,27.15,36.35,False,9.2
3,3,L53355,0.016727,1524,44.3,197,0,0,0,0,0,0,67513.2,28.85,38.75,False,9.9
4,4,M24050,0.013592,1641,35.4,34,0,0,0,0,0,0,58091.4,25.85,36.85,False,11.0


In [106]:
pd.DataFrame(index=[df_train.groupby('Tool wear [min]')['Machine failure']])

""


In [ ]:
pd.DataFrame(index=[df_train.groupby('Tool wear [min]')['Machine failure'].mean(),
                   df_train['Product ID'].unique()])

In [120]:
pd.DataFrame(index=[df_train.groupby('Tool wear [min]')['Machine failure'].mean(),
                   df_train['Product ID'].unique()])

ValueError: all arrays must be same length

In [24]:
df_train.head()

,id,Product ID,Type,Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF,Power [KW],Air temperature [C],Process temperature [C],Possbile_overheating,Temperature difference
0,0,L50096,0.016727,1596,36.1,140,0,0,0,0,0,0,57615.6,28.45,37.45,True,-9.0
1,1,M20343,0.013592,1759,29.1,200,0,0,0,0,0,0,51186.9,30.45,39.95,True,-9.5
2,2,L49454,0.016727,1805,26.5,25,0,0,0,0,0,0,47832.5,27.15,36.35,True,-9.2
3,3,L53355,0.016727,1524,44.3,197,0,0,0,0,0,0,67513.2,28.85,38.75,True,-9.9
4,4,M24050,0.013592,1641,35.4,34,0,0,0,0,0,0,58091.4,25.85,36.85,True,-11.0


##Pochwalić się Kacprowi ;3


In [48]:
pd.DataFrame(df_train.groupby('Tool wear [min]')['Machine failure'].mean()).describe()

,Machine failure
count,246.000000
mean,0.034653
std,0.085217
min,0.000000
25%,0.006726
50%,0.011525
75%,0.023674
max,1.000000


In [49]:
pd.DataFrame(df_train.groupby('Rotational speed [rpm]')['Machine failure'].mean()).describe()

,Machine failure
count,952.000000
mean,0.056718
std,0.168048
min,0.000000
25%,0.000000
50%,0.000000
75%,0.015444
max,1.000000
